In [ ]:
"""
서울시 상권분석서비스 - 상권별 매출 정보 API 호출
- OA-15572
- 서비스명: VwsmTrdarSelngQq
- 편의점(CS300002)만 필터링
- 기준년분기 + 상권유형별 매출 집계
+ EDA (탐색적 데이터 분석) 포함
"""

import requests
import pandas as pd
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import sys

# ============================================================
# Jupyter 출력 설정 (잘림 방지)
# ============================================================
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
# plt.rcParams['font.family'] = 'AppleGothic'  # Mac
plt.rcParams['axes.unicode_minus'] = False

# ============================================================
# 설정
# ============================================================
API_KEY = "455963584d68666f32367441675145"
SERVICE_NAME = "VwsmTrdarSelngQq"
BASE_URL = f"http://openapi.seoul.go.kr:8088/{API_KEY}/json/{SERVICE_NAME}"

# 편의점 업종코드
CONVENIENCE_STORE_CODE = "CS300002"

def generate_quarter_codes(start_year=2022, end_year=2025):
    codes = []
    for year in range(start_year, end_year + 1):
        for quarter in range(1, 5):
            codes.append(f"{year}{quarter}")
    return codes

QUARTER_CODES = generate_quarter_codes(2022, 2025)
print(f"수집 대상 분기: {QUARTER_CODES}")
print(f"총 {len(QUARTER_CODES)}개 분기")
print(f"필터링 업종: 편의점 ({CONVENIENCE_STORE_CODE})")
sys.stdout.flush()


# ============================================================
# API 호출 함수
# ============================================================
def fetch_all_data():
    """전체 상권별 매출 데이터 수집"""
    all_data = []
    start = 1
    batch_size = 1000
    max_retries = 3
    max_iterations = 1000
    iteration = 0
    
    print(f"\n{'='*60}")
    print("서울시 상권분석 API - 상권별 매출 데이터 수집 시작")
    print(f"{'='*60}")
    sys.stdout.flush()
    
    while iteration < max_iterations:
        iteration += 1
        end = start + batch_size - 1
        url = f"{BASE_URL}/{start}/{end}"
        
        for retry in range(max_retries):
            try:
                response = requests.get(url, timeout=60)
                response.raise_for_status()
                result = response.json()
                
                if SERVICE_NAME not in result:
                    if 'RESULT' in result:
                        error_msg = result.get('RESULT', {})
                        print(f"API 에러: {error_msg}")
                        if 'INFO-200' in str(error_msg):
                            print("데이터 수집 완료 (더 이상 데이터 없음)")
                            return all_data
                    return all_data
                
                code = result[SERVICE_NAME]['RESULT']['CODE']
                if code != 'INFO-000':
                    print(f"API 응답 코드: {code}")
                    if code == 'INFO-200':
                        return all_data
                    break
                
                rows = result[SERVICE_NAME].get('row', [])
                if not rows:
                    print("더 이상 데이터가 없습니다.")
                    return all_data
                
                all_data.extend(rows)
                total_count = result[SERVICE_NAME]['list_total_count']
                
                if len(all_data) % 10000 < batch_size:
                    print(f"수집 중: {start:,}~{end:,} / 전체 {total_count:,}건 (현재 {len(all_data):,}건)")
                    sys.stdout.flush()
                
                if len(all_data) >= total_count:
                    print(f"\n✅ 전체 데이터 수집 완료: {len(all_data):,}건")
                    return all_data
                
                start += batch_size
                time.sleep(0.2)
                break
                
            except requests.exceptions.Timeout:
                print(f"⏰ 타임아웃 (시도 {retry+1}/{max_retries}): {start}~{end}")
                sys.stdout.flush()
                time.sleep(2)
                continue
                
            except requests.exceptions.RequestException as e:
                print(f"❌ 요청 에러 (시도 {retry+1}/{max_retries}): {e}")
                sys.stdout.flush()
                time.sleep(2)
                continue
                
            except Exception as e:
                print(f"❌ 처리 에러: {e}")
                sys.stdout.flush()
                return all_data
        
        else:
            print(f"❌ {start}~{end} 구간 수집 실패, 다음으로 진행")
            start += batch_size
            continue
    
    print(f"⚠️ 최대 반복 횟수 도달 ({max_iterations}회)")
    return all_data


def filter_convenience_store(data, quarter_codes):
    """편의점만 필터링 + 기준년분기 필터링"""
    df = pd.DataFrame(data)
    print(f"\n전체 수집 데이터: {len(df):,}건")
    
    # 분기 필터링
    df['STDR_YYQU_CD'] = df['STDR_YYQU_CD'].astype(str)
    df = df[df['STDR_YYQU_CD'].isin(quarter_codes)]
    print(f"2022~2025년 데이터: {len(df):,}건")
    
    # 편의점 필터링
    df = df[df['SVC_INDUTY_CD'] == CONVENIENCE_STORE_CODE]
    print(f"편의점 데이터: {len(df):,}건")
    
    return df


# ============================================================
# 컬럼명 한글 변환
# ============================================================
COLUMN_MAPPING = {
    'STDR_YYQU_CD': '기준_년분기_코드',
    'TRDAR_SE_CD': '상권구분_코드',
    'TRDAR_SE_CD_NM': '상권구분_명',
    'TRDAR_CD': '상권_코드',
    'TRDAR_CD_NM': '상권_명',
    'SVC_INDUTY_CD': '서비스_업종_코드',
    'SVC_INDUTY_CD_NM': '서비스_업종_명',
    'THSMON_SELNG_AMT': '당월_매출_금액',
    'THSMON_SELNG_CO': '당월_매출_건수',
    'MDWK_SELNG_AMT': '주중_매출_금액',
    'WKEND_SELNG_AMT': '주말_매출_금액',
    'MON_SELNG_AMT': '월요일_매출_금액',
    'TUES_SELNG_AMT': '화요일_매출_금액',
    'WED_SELNG_AMT': '수요일_매출_금액',
    'THUR_SELNG_AMT': '목요일_매출_금액',
    'FRI_SELNG_AMT': '금요일_매출_금액',
    'SAT_SELNG_AMT': '토요일_매출_금액',
    'SUN_SELNG_AMT': '일요일_매출_금액',
    'ML_SELNG_AMT': '남성_매출_금액',
    'FML_SELNG_AMT': '여성_매출_금액',
}


# ============================================================
# 기준년분기 + 상권유형별 집계 함수
# ============================================================
def aggregate_by_quarter_type(df):
    """
    기준년분기 + 상권유형별 편의점 매출 집계
    """
    print(f"\n{'='*60}")
    print("📊 기준년분기 + 상권유형별 편의점 매출 집계")
    print(f"{'='*60}")
    
    agg_df = df.groupby(['기준_년분기_코드', '상권구분_코드', '상권구분_명']).agg(
        총_매출_금액=('당월_매출_금액', 'sum'),
        총_매출_건수=('당월_매출_건수', 'sum'),
        평균_매출_금액=('당월_매출_금액', 'mean'),
        상권_수=('상권_코드', 'nunique'),
        주중_매출_금액=('주중_매출_금액', 'sum'),
        주말_매출_금액=('주말_매출_금액', 'sum'),
    ).reset_index()
    
    agg_df['건당_매출_금액'] = (agg_df['총_매출_금액'] / agg_df['총_매출_건수']).round(0)
    
    print(f"집계 완료: {len(agg_df)}행 (분기 x 상권유형)")
    
    return agg_df


# ============================================================
# EDA 함수
# ============================================================
def run_eda(df_raw, df_agg):
    """
    탐색적 데이터 분석 (EDA) - 편의점 매출
    """
    print(f"\n{'='*60}")
    print("📊 EDA (탐색적 데이터 분석) 시작 - 편의점 매출")
    print(f"{'='*60}")
    sys.stdout.flush()
    
    # =========================================================
    # 1단계: 데이터 훑어보기
    # =========================================================
    print(f"\n{'='*60}")
    print("📌 1단계: 데이터 훑어보기")
    print(f"{'='*60}")
    
    print("\n[1-1] 데이터 크기")
    print("-" * 40)
    print(f"전체 데이터: {len(df_raw):,}건")
    print(f"분기 수: {df_raw['기준_년분기_코드'].nunique()}개")
    print(f"상권 수: {df_raw['상권_코드'].nunique()}개")
    
    print("\n[1-2] 상권유형별 데이터 건수")
    print("-" * 40)
    상권유형_분포 = df_raw['상권구분_명'].value_counts()
    for 유형, 개수 in 상권유형_분포.items():
        print(f"  - {유형}: {개수:,}건 ({개수/len(df_raw)*100:.1f}%)")
    
    print("\n[1-3] 상위 5개 행 미리보기")
    print("-" * 40)
    print(df_raw[['기준_년분기_코드', '상권구분_명', '상권_명', '당월_매출_금액']].head())
    
    print("\n[1-4] 결측치 확인")
    print("-" * 40)
    missing = df_raw.isnull().sum()
    missing_with_values = missing[missing > 0]
    if len(missing_with_values) > 0:
        print(missing_with_values)
    else:
        print("✅ 결측치 없음!")
    
    sys.stdout.flush()
    
    # =========================================================
    # 2단계: 숫자 요약하기
    # =========================================================
    print(f"\n{'='*60}")
    print("📌 2단계: 숫자 요약하기")
    print(f"{'='*60}")
    
    print("\n[2-1] 집계 데이터 미리보기")
    print("-" * 40)
    print(df_agg.head(10))
    
    print("\n[2-2] 상권유형별 총 매출 (전체 기간)")
    print("-" * 40)
    type_total = df_agg.groupby('상권구분_명')['총_매출_금액'].sum().sort_values(ascending=False)
    for 유형, 금액 in type_total.items():
        print(f"  - {유형}: {금액/1e12:.2f}조원")
    
    print("\n[2-3] 기초 통계량 (당월_매출_금액)")
    print("-" * 40)
    stats = df_raw['당월_매출_금액'].describe()
    print(f"개수: {stats['count']:,.0f}건")
    print(f"평균: {stats['mean']/1e6:,.1f}백만원")
    print(f"최소: {stats['min']/1e6:,.1f}백만원")
    print(f"최대: {stats['max']/1e9:,.2f}십억원")
    print(f"중앙값: {stats['50%']/1e6:,.1f}백만원")
    
    sys.stdout.flush()
    
    # =========================================================
    # 3단계: 그래프로 보기
    # =========================================================
    print(f"\n{'='*60}")
    print("📌 3단계: 그래프로 보기")
    print(f"{'='*60}")
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 3-1. 상권유형별 총 매출 (파이차트)
    colors = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99']
    axes[0, 0].pie(type_total.values, labels=type_total.index, autopct='%1.1f%%', colors=colors[:len(type_total)])
    axes[0, 0].set_title('상권유형별 편의점 총 매출 비중', fontsize=12)
    
    # 3-2. 분기별 총 매출 추이
    quarterly = df_agg.groupby('기준_년분기_코드')['총_매출_금액'].sum() / 1e12
    quarterly.plot(kind='bar', ax=axes[0, 1], color='steelblue', edgecolor='black')
    axes[0, 1].set_title('분기별 편의점 총 매출 추이', fontsize=12)
    axes[0, 1].set_xlabel('분기')
    axes[0, 1].set_ylabel('총 매출 (조원)')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # 3-3. 상권유형별 분기 매출 추이 (선그래프)
    for 유형 in df_agg['상권구분_명'].unique():
        data = df_agg[df_agg['상권구분_명'] == 유형].set_index('기준_년분기_코드')['총_매출_금액'] / 1e12
        axes[1, 0].plot(data.index, data.values, marker='o', label=유형)
    axes[1, 0].set_title('상권유형별 편의점 분기 매출 추이', fontsize=12)
    axes[1, 0].set_xlabel('분기')
    axes[1, 0].set_ylabel('총 매출 (조원)')
    axes[1, 0].legend()
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # 3-4. 매출 분포 (박스플롯 - 상권유형별)
    df_raw.boxplot(column='당월_매출_금액', by='상권구분_명', ax=axes[1, 1])
    axes[1, 1].set_title('상권유형별 편의점 매출 분포', fontsize=12)
    axes[1, 1].set_xlabel('상권유형')
    axes[1, 1].set_ylabel('매출 (원)')
    plt.suptitle('')
    
    plt.tight_layout()
    plt.savefig('eda_편의점매출_시각화.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("✅ 시각화 저장 완료: eda_편의점매출_시각화.png")
    
    sys.stdout.flush()
    
    # =========================================================
    # 4단계: 이상한 점 발견하기
    # =========================================================
    print(f"\n{'='*60}")
    print("📌 4단계: 이상한 점 발견하기")
    print(f"{'='*60}")
    
    # 변수 초기화
    zero_sales = pd.DataFrame()
    negative_sales = pd.DataFrame()
    outliers_high = pd.DataFrame()
    total_outliers = 0
    
    # ---------------------------------------------------------
    # 4-1. 매출 0원 데이터
    # ---------------------------------------------------------
    print("\n[4-1] 매출 0원 데이터")
    print("-" * 40)
    zero_sales = df_raw[df_raw['당월_매출_금액'] == 0]
    print(f"매출 0원 데이터: {len(zero_sales):,}건 ({len(zero_sales)/len(df_raw)*100:.2f}%)")
    
    if len(zero_sales) > 0:
        print("\n📍 매출 0원 - 상권유형별 분포:")
        zero_by_type = zero_sales['상권구분_명'].value_counts()
        for 유형, 개수 in zero_by_type.items():
            print(f"  - {유형}: {개수:,}건")
        
        zero_sales[['기준_년분기_코드', '상권구분_명', '상권_명', '당월_매출_금액']].to_csv(
            'eda_편의점_매출0원.csv', index=False, encoding='utf-8-sig')
        print(f"✅ CSV 저장 완료: eda_편의점_매출0원.csv")
    
    # ---------------------------------------------------------
    # 4-2. 매출 음수 데이터
    # ---------------------------------------------------------
    print("\n[4-2] 매출 음수 데이터")
    print("-" * 40)
    negative_sales = df_raw[df_raw['당월_매출_금액'] < 0]
    print(f"매출 음수 데이터: {len(negative_sales):,}건")
    
    # ---------------------------------------------------------
    # 4-3. 이상치 확인 (IQR 방식)
    # ---------------------------------------------------------
    print("\n[4-3] 이상치 확인 (당월_매출_금액 - IQR 방식)")
    print("-" * 40)
    
    q1 = df_raw['당월_매출_금액'].quantile(0.25)
    q3 = df_raw['당월_매출_금액'].quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    
    print(f"\n📐 IQR 이상치 판별 기준:")
    print(f"  - Q1 (25%): {q1/1e6:,.1f}백만원")
    print(f"  - Q3 (75%): {q3/1e6:,.1f}백만원")
    print(f"  - IQR: {iqr/1e6:,.1f}백만원")
    print(f"  - 하한 경계: {lower/1e6:,.1f}백만원")
    print(f"  - 상한 경계: {upper/1e6:,.1f}백만원")
    
    outliers_low = df_raw[df_raw['당월_매출_금액'] < lower]
    outliers_high = df_raw[df_raw['당월_매출_금액'] > upper]
    
    print(f"\n[하한 이상치] {len(outliers_low):,}건")
    print(f"[상한 이상치] {len(outliers_high):,}건 ({len(outliers_high)/len(df_raw)*100:.1f}%)")
    
    if len(outliers_high) > 0:
        print("\n⚠️ 상한 이상치 TOP 10 (매출 가장 높은 편의점 상권):")
        top10 = outliers_high.nlargest(10, '당월_매출_금액')[['상권구분_명', '상권_명', '기준_년분기_코드', '당월_매출_금액']]
        for idx, row in top10.iterrows():
            print(f"  - {row['상권_명']} ({row['상권구분_명']}, {row['기준_년분기_코드']}): {row['당월_매출_금액']/1e8:.1f}억원")
        
        # 이상치 빈도 TOP 10
        print("\n📍 상한 이상치 빈도 TOP 10 (어떤 상권이 자주 이상치로 잡히나):")
        freq_high = outliers_high.groupby('상권_명').agg(
            이상치_횟수=('당월_매출_금액', 'count'),
            평균_매출=('당월_매출_금액', 'mean')
        ).sort_values('이상치_횟수', ascending=False).head(10)
        for 상권, row in freq_high.iterrows():
            print(f"  - {상권}: {row['이상치_횟수']}건 (평균 {row['평균_매출']/1e8:.1f}억원)")
        
        outliers_high_save = outliers_high[['기준_년분기_코드', '상권구분_명', '상권_명', '당월_매출_금액']].copy()
        outliers_high_save['매출_억원'] = (outliers_high_save['당월_매출_금액'] / 1e8).round(1)
        outliers_high_save = outliers_high_save.sort_values('당월_매출_금액', ascending=False)
        outliers_high_save.to_csv('eda_편의점_상한이상치.csv', index=False, encoding='utf-8-sig')
        print(f"\n✅ CSV 저장 완료: eda_편의점_상한이상치.csv ({len(outliers_high)}건)")
    
    total_outliers = len(outliers_low) + len(outliers_high)
    print(f"\n📊 이상치 요약:")
    print(f"  - 하한 이상치: {len(outliers_low):,}건")
    print(f"  - 상한 이상치: {len(outliers_high):,}건")
    print(f"  - 전체 이상치: {total_outliers:,}건 ({total_outliers/len(df_raw)*100:.1f}%)")
    
    # ---------------------------------------------------------
    # 4-4. 분기별 데이터 건수 확인
    # ---------------------------------------------------------
    print("\n[4-4] 분기별 데이터 건수 확인")
    print("-" * 40)
    quarter_counts = df_raw.groupby('기준_년분기_코드').size().sort_index()
    print(quarter_counts)
    
    avg_count = quarter_counts.mean()
    print(f"\n분기 평균 데이터 수: {avg_count:,.0f}건")
    
    abnormal_quarters = quarter_counts[abs(quarter_counts - avg_count) > avg_count * 0.2]
    if len(abnormal_quarters) > 0:
        print("\n⚠️ 평균 대비 ±20% 이상 차이나는 분기:")
        for q, cnt in abnormal_quarters.items():
            diff_pct = (cnt - avg_count) / avg_count * 100
            print(f"  - {q}: {cnt}건 ({diff_pct:+.1f}%)")
    
    sys.stdout.flush()
    
    # =========================================================
    # EDA 요약
    # =========================================================
    print(f"\n{'='*60}")
    print("📊 EDA 요약 - 편의점 매출")
    print(f"{'='*60}")
    print(f"전체 데이터: {len(df_raw):,}건")
    print(f"분기 수: {df_raw['기준_년분기_코드'].nunique()}개")
    print(f"상권 수: {df_raw['상권_코드'].nunique()}개")
    print(f"\n상권유형별 편의점 총 매출:")
    for 유형, 금액 in type_total.items():
        print(f"  - {유형}: {금액/1e12:.2f}조원")
    
    issues = []
    if len(zero_sales) > 0:
        issues.append(f"매출 0원: {len(zero_sales):,}건")
    if len(negative_sales) > 0:
        issues.append(f"매출 음수: {len(negative_sales):,}건")
    if total_outliers > 0:
        issues.append(f"이상치: {total_outliers:,}건")
    
    if issues:
        print(f"\n⚠️ 발견된 이슈:")
        for issue in issues:
            print(f"  - {issue}")
    else:
        print("\n✅ 특별한 이슈 없음!")
    
    print(f"\n📁 저장된 CSV 파일 목록:")
    print("  - 서울시_편의점매출_원본.csv (전체 데이터)")
    print("  - 서울시_편의점매출_분기별_상권유형별.csv (집계 데이터)")
    if len(zero_sales) > 0:
        print("  - eda_편의점_매출0원.csv")
    if len(outliers_high) > 0:
        print("  - eda_편의점_상한이상치.csv")
    
    print(f"\n{'='*60}")
    print("📊 EDA 완료!")
    print(f"{'='*60}")
    
    return df_agg


# ============================================================
# 메인 실행
# ============================================================
if __name__ == "__main__":
    print("🚀 프로그램 시작")
    print(f"현재 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("⚠️ 데이터가 57만건이라 수집에 10~15분 소요될 수 있습니다.")
    print("   (편의점만 필터링하면 약 1~2만건 예상)")
    sys.stdout.flush()
    
    # 1. 데이터 수집
    raw_data = fetch_all_data()
    
    if raw_data:
        # 2. 편의점 필터링 + 기간 필터링
        df_raw = filter_convenience_store(raw_data, QUARTER_CODES)
        
        if len(df_raw) > 0:
            # 3. 컬럼명 한글 변환
            existing_cols = {k: v for k, v in COLUMN_MAPPING.items() if k in df_raw.columns}
            df_raw = df_raw.rename(columns=existing_cols)
            
            # 4. 숫자형 변환
            numeric_cols = [col for col in df_raw.columns if '매출' in col]
            for col in numeric_cols:
                df_raw[col] = pd.to_numeric(df_raw[col], errors='coerce')
            
            # 5. 필요한 컬럼만 선택
            keep_cols = ['기준_년분기_코드', '상권구분_코드', '상권구분_명', '상권_코드', '상권_명',
                         '당월_매출_금액', '당월_매출_건수', '주중_매출_금액', '주말_매출_금액']
            df_raw = df_raw[[col for col in keep_cols if col in df_raw.columns]]
            
            # 6. 원본 데이터 저장
            df_raw.to_csv('서울시_편의점매출_원본.csv', index=False, encoding='utf-8-sig')
            print(f"\n✅ 저장 완료: 서울시_편의점매출_원본.csv ({len(df_raw):,}건)")
            
            # 7. 분기별 + 상권유형별 집계
            df_agg = aggregate_by_quarter_type(df_raw)
            
            # 8. 집계 데이터 저장
            df_agg.to_csv('서울시_편의점매출_분기별_상권유형별.csv', index=False, encoding='utf-8-sig')
            print(f"✅ 저장 완료: 서울시_편의점매출_분기별_상권유형별.csv ({len(df_agg)}건)")
            
            # 9. EDA 실행
            df_agg = run_eda(df_raw, df_agg)
            
        else:
            print("편의점 데이터가 없습니다.")
    else:
        print("데이터 수집 실패")
    
    print(f"\n🏁 프로그램 종료: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

수집 대상 분기: ['20221', '20222', '20223', '20224', '20231', '20232', '20233', '20234', '20241', '20242', '20243', '20244', '20251', '20252', '20253', '20254']
총 16개 분기
필터링 업종: 편의점 (CS300002)
🚀 프로그램 시작
현재 시간: 2026-01-25 15:49:12
⚠️ 데이터가 57만건이라 수집에 10~15분 소요될 수 있습니다.
   (편의점만 필터링하면 약 1~2만건 예상)

서울시 상권분석 API - 상권별 매출 데이터 수집 시작
